In [1]:
!/opt/conda/bin/pip uninstall -y chromadb
!/opt/conda/bin/pip install chromadb
!/opt/conda/bin/pip install sentence_transformers

Found existing installation: chromadb 0.6.3
Uninstalling chromadb-0.6.3:
  Successfully uninstalled chromadb-0.6.3
Defaulting to user installation because normal site-packages is not writeable
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
import chromadb
from chromadb.utils import embedding_functions

In [3]:
emb_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name='Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

In [4]:
chroma_client = chromadb.PersistentClient(path='./DBV')

In [5]:
collection = chroma_client.get_collection(name="mails_enron", embedding_function=emb_fn)

In [6]:
total_count = collection.count()
print(f"Nombre total de documents dans la collection : {total_count}")

Nombre total de documents dans la collection : 517401


In [7]:
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1/chat/completions"
headers = {
    "Authorization": "Bearer nothing",
    "Content-Type": "application/json"
}

In [9]:
while True:
    user_input = input("Entrez une question/requête (ou 'x' pour quitter) : ").strip()

    if user_input.lower() == "x":
        print("Recherche documentaire terminé.")
        break

    # Effectuer la requête pour récupérer les mails pertinents
    mails = collection.query(query_texts=[user_input], n_results=15)

    # Vérifier si des mails ont été trouvés
    if "ids" not in mails or "documents" not in mails:
        print("Aucun résultat trouvé.")
        continue

    # Construction du contexte
    context = ""
    mail_ids = mails.get("ids", [[]])[0]  # Liste des IDs
    mail_docs = mails.get("documents", [[]])[0]  # Liste des documents

    for mail_id, mail_doc in zip(mail_ids, mail_docs):
        context += f"Mail ID: {mail_id}\nContenu:\n{mail_doc}\n\n"

    if not context.strip():
        print("Aucun contenu exploitable trouvé dans les mails.")
        continue

    # Préparer la requête JSON
    payload = {
        "model": "casperhansen/llama-3-70b-instruct-awq",  # Nom du modèle
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that answers user questions based on the context provided. If the answer is not specified in the context, say 'I don't know the answer, it is not in the context'."},
            {"role": "user", "content": f"user question: {user_input}"},
            {"role": "user", "content": f"context: {context}"}
        ]
    }

    try:
        # Envoyer la requête
        response = requests.post(url, headers=headers, json=payload)

        # Vérifier la réponse
        if response.status_code == 200:
            response_json = response.json()
            message_content = response_json.get("choices", [{}])[0].get("message", {}).get("content", "Réponse introuvable.")
            print("\nRéponse :", message_content)
        else:
            print(f"Erreur API ({response.status_code}) : {response.text}")

    except requests.exceptions.RequestException as e:
        print("Erreur lors de la requête :", str(e))


Entrez une question/requête (ou 'x' pour quitter) :  How can one request authorization to use eConnect?



Réponse : One can request authorization to use eConnect via the eRequest system, which can be found on http://itcentral.enron.com.


Entrez une question/requête (ou 'x' pour quitter) :  What are the concerns regarding the receipt meter on deal 235367?



Réponse : The concern regarding the receipt meter on deal 235367 is that it cannot be changed from meter 7126 to meter 010902.


Entrez une question/requête (ou 'x' pour quitter) :  x


Recherche documentaire terminé.
